IBM HR의 퇴직률 데이터를 이용하여, 결정트리를 만들자. 결정트리를 이용하여 어떤 사람이 퇴직을 여부를 분류할 수 있는 분류기를 만들자.

In [3]:
import pandas as pd
pd.options.display.max_columns=None

In [5]:
datasets = pd.read_csv('./inputs/HR-Employee-Attrition.csv')
datasets.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,Y,Yes,15,3,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,4,Female,56,3,1,Research Scientist,3,Married,2909,23159,1,Y,Yes,11,3,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468,16632,9,Y,No,12,3,4,80,1,6,3,3,2,2,2,2


Taget variable은 무엇인가?

In [10]:
# 계산을 위하여, 0, 1의 id로 부류를 표현하자.
datasets['Attrition_idx'] = datasets['Attrition'].apply(lambda x: 1 if x == 'Yes' else 0)
datasets.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition_idx
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5,1
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7,0
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,Y,Yes,15,3,2,80,0,7,3,3,0,0,0,0,1
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,4,Female,56,3,1,Research Scientist,3,Married,2909,23159,1,Y,Yes,11,3,3,80,0,8,3,3,8,7,3,0,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468,16632,9,Y,No,12,3,4,80,1,6,3,3,2,2,2,2,0


Column을 살펴보자.

In [87]:
col_names = datasets.columns
col_names

Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager', 'Attrition_idx'],
      dtype='object')

필요없는 변수들이 있다 : *EmployeeCount*, *EmployeeNumber*, *Over18*, *StandardHours*

In [90]:
print(datasets.Over18.value_counts())
print(datasets.EmployeeCount.value_counts())
print(datasets.StandardHours.value_counts())

Y    1470
Name: Over18, dtype: int64
1    1470
Name: EmployeeCount, dtype: int64
80    1470
Name: StandardHours, dtype: int64


In [88]:
# target은 feature에서 제외한다.
col_names = col_names.drop([target, 'Attrition', 'Over18', 'EmployeeCount', 'EmployeeNumber', 'StandardHours'])

Categorical column을 다루어보자.

Catagorical column을 numerical column을 나누어보자.

In [92]:
categorical_features = []
numerical_features = []
target = 'Attrition_idx'

# feature를 2가지 형태로 구분한다.
for col in col_names:
    if datasets[col].dtype == 'O':
        categorical_features.append(col)
    else:
        numerical_features.append(col)

In [93]:
print('Categorical feature의 수 :', len(categorical_features))
print('Numerical feature의 수 :', len(numerical_features))

Categorical feature의 수 : 7
Numerical feature의 수 : 23


In [94]:
categorical_features

['BusinessTravel',
 'Department',
 'EducationField',
 'Gender',
 'JobRole',
 'MaritalStatus',
 'OverTime']

In [95]:
numerical_features[:5]

['Age',
 'DailyRate',
 'DistanceFromHome',
 'Education',
 'EnvironmentSatisfaction']

Categorical 데이터를 One-hot vector로 변경하자. Pandas에서 `get_dummies`를 이용하자.

In [98]:
categorical_datasets = pd.get_dummies(datasets[categorical_features])
categorical_datasets.head()

,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,Gender_Female,Gender_Male,JobRole_Healthcare Representative,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_No,OverTime_Yes
0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1
1,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1,0
2,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1
3,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1
4,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0


In [100]:
numerical_datasets = datasets[numerical_features]
numerical_datasets.head()

,Age,DailyRate,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,MonthlyRate,NumCompaniesWorked,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,1102,1,2,2,94,3,2,4,5993,19479,8,11,3,1,0,8,0,1,6,4,0,5
1,49,279,8,1,3,61,2,2,2,5130,24907,1,23,4,4,1,10,3,3,10,7,1,7
2,37,1373,2,2,4,92,2,1,3,2090,2396,6,15,3,2,0,7,3,3,0,0,0,0
3,33,1392,3,4,4,56,3,1,3,2909,23159,1,11,3,3,0,8,3,3,8,7,3,0
4,27,591,2,1,1,40,3,1,2,3468,16632,9,12,3,4,1,6,3,3,2,2,2,2


Categorical dataset과 numerical dataset을 합친다. 모델의 input으로 사용할 X이다.

In [101]:
X = pd.concat([categorical_datasets, numerical_datasets], axis=1)
X.head()

,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,Gender_Female,Gender_Male,JobRole_Healthcare Representative,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_No,OverTime_Yes,Age,DailyRate,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,MonthlyRate,NumCompaniesWorked,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,41,1102,1,2,2,94,3,2,4,5993,19479,8,11,3,1,0,8,0,1,6,4,0,5
1,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1,0,49,279,8,1,3,61,2,2,2,5130,24907,1,23,4,4,1,10,3,3,10,7,1,7
2,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,37,1373,2,2,4,92,2,1,3,2090,2396,6,15,3,2,0,7,3,3,0,0,0,0
3,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,33,1392,3,4,4,56,3,1,3,2909,23159,1,11,3,3,0,8,3,3,8,7,3,0
4,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,27,591,2,1,1,40,3,1,2,3468,16632,9,12,3,4,1,6,3,3,2,2,2,2


In [102]:
y = datasets[target]
y.head()

0    1
1    0
2    1
3    0
4    0
Name: Attrition_idx, dtype: int64

Train set과 test set을 분할한다.

In [103]:
from sklearn.model_selection import train_test_split

In [107]:
x_train, x_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.3, random_state=42)

Decision tree를 이용하여 분류기를 생성한다.
 - 트리의 최대깊이 : 5
 - 분할을 위한 최소 관측값 : 2
 - 각 단말 도느에서 필요한 최소 관측 값 : 1

In [108]:
from sklearn.tree import DecisionTreeClassifier

In [110]:
# 학습기 생성
tree_classifier = DecisionTreeClassifier(
    criterion='gini',
    max_depth=5,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42
)

# 학습
tree_classifier.fit(x_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [124]:
pred_train = tree_classifier.predict(x_train)
pred_test = tree_classifier.predict(x_test)

In [123]:
from sklearn.metrics import accuracy_score, classification_report

In [133]:
# 학습셋에 대한 결과값을 살펴보자
# 1. Confusion Matrix
print('\n Train Confusion Matrix :')
display(pd.crosstab(y_train, pred_train, rownames=['Actual'], colnames=['Predict']))

# 2. Accuracy
print('\n Train accuracy :', accuracy_score(y_train, pred_train))

# 3. Classification Report
print('\n Classification Report : \n', classification_report(y_train, pred_train))


 Train Confusion Matrix :


Predict,0,1
Actual,,
0,844,9
1,98,78



 Train accuracy : 0.8960155490767736

 Classification Report : 
              precision    recall  f1-score   support

          0       0.90      0.99      0.94       853
          1       0.90      0.44      0.59       176

avg / total       0.90      0.90      0.88      1029



In [135]:
# 테스트셋에 대한 결과값을 살펴보자
# 1. Confusion Matrix
print('\n Test Confusion Matrix :')
display(pd.crosstab(y_test, pred_test, rownames=['Actual'], colnames=['Predict']))

# 2. Accuracy
print('\n Test accuracy :', accuracy_score(y_test, pred_test))

# 3. Classification Report
print('\n Classification Report : \n', classification_report(y_test, pred_test))


 Test Confusion Matrix :


Predict,0,1
Actual,,
0,361,19
1,49,12



 Test accuracy : 0.8458049886621315

 Classification Report : 
              precision    recall  f1-score   support

          0       0.88      0.95      0.91       380
          1       0.39      0.20      0.26        61

avg / total       0.81      0.85      0.82       441



In [152]:
# 나중에 결과 테이블을 만들 떄 사용
print(classification_report(y_test, pred_test).split())

['precision', 'recall', 'f1-score', 'support', '0', '0.88', '0.95', '0.91', '380', '1', '0.39', '0.20', '0.26', '61', 'avg', '/', 'total', '0.81', '0.85', '0.82', '441']


정확도가 높은 것은 그렇게 의미가 없다.

In [138]:
datasets.Attrition_idx.value_counts()

0    1233
1     237
Name: Attrition_idx, dtype: int64

In [140]:
(1233-237)/1233

0.8077858880778589

값을 살펴보면 1의 비율이 6:1이다. 따라서 분류기가 모든 샘플에 대하여 0이라고만 분류해도, 80.77%의 정확도를 얻을 수 있다. 1(퇴직자)에 대한 분류를 제대로 못하고 있다.

퇴사할 가능성이 많은 직원에게 보너스를 많이 주어, 퇴사를 방지하는 것에 이 모델을 사용한다면, 심각한 문제가 될 수 있다. 모델은 퇴직하지 않는다고 예측했는데, 예측과 달리 퇴직한 직원의 비율이 상당히 높다.

모델을 살짝 튜닝해보자. 부류의 가중값을 조절해보자. 예를 들어, 부류 1(퇴직자)의 가중값을 올리면 실제로 퇴사할 특성이 있는 직원들을 더 잘 파악하게 되지만, 퇴사할 가능성이 없는 일부 직원들을 잠재적 퇴사자로 분류하게 된다. 즉, 퇴사를 더 잘 막을 수 있게 될 것이다. (대출을 실행할 때, 신용도 낮은 사람을 승인하는 것보다, 신용도가 조금 만족되는 사람이라도 거절하는 편이 더 낫다. 즉 감당할 수 있는 오류이다.)

부류의 가중값을 바꾸어보며 테스트 한다.

In [156]:
import numpy as np

In [157]:
tuning_results = pd.DataFrame(np.empty((6, 10)))

In [158]:
tuning_results.columns = ['class_0_weight', 'class_1_weight', 
                          'train_accuracy', 'tesst_accuracy', 
                          'precision_class_0', 'precision_class_1', 'precision_overall', 
                          'recall_calss_0', 'recall_class_1', 'recall_overall']

In [176]:
class_0_weight = [0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.99]

for i in range(len(class_0_weight)):
    class_weights = {0: class_0_weight[i], 1: 1 - class_0_weight[i]}
    tree_classifier = DecisionTreeClassifier(criterion='gini',
                                             max_depth=5,
                                             min_samples_split=2,
                                             min_samples_leaf=1,
                                             random_state=42,
                                             class_weight=class_weights)
    tree_classifier.fit(x_train, y_train)
    pred_train = tree_classifier.predict(x_train)
    pred_test = tree_classifier.predict(x_test)
    tuning_results.loc[i, 'class_0_weight'] = class_weights[0]
    tuning_results.loc[i, 'class_1_weight'] = class_weights[1]
    tuning_results.loc[i, 'train_accuracy'] = round(accuracy_score(y_train, pred_train), 4)
    tuning_results.loc[i, 'test_accuracy'] = round(accuracy_score(y_test, pred_test), 4)
    c_r = classification_report(y_test, pred_test).split()
    tuning_results.loc[i, 'precision_class_0'] = float(c_r[5])
    tuning_results.loc[i, 'precision_class_1'] = float(c_r[10])
    tuning_results.loc[i, 'precision_overall'] = float(c_r[17])
    tuning_results.loc[i, 'recall_calss_0'] = float(c_r[6])
    tuning_results.loc[i, 'recall_class_1'] = float(c_r[11])
    tuning_results.loc[i, 'recall_overall'] = float(c_r[18])

    print(class_weights)
    print('Test accuracy :', accuracy_score(y_test, pred_test))
    display(pd.crosstab(y_test, pred_test, rownames=['Actual'], colnames=['Predict']))

{0: 0.01, 1: 0.99}
Test accuracy : 0.272108843537415


Predict,0,1
Actual,,
0,65,315
1,6,55


{0: 0.1, 1: 0.9}
Test accuracy : 0.7324263038548753


Predict,0,1
Actual,,
0,282,98
1,20,41


{0: 0.2, 1: 0.8}
Test accuracy : 0.8299319727891157


...

Predict,0,1
Actual,,
0,370,10
1,55,6


{0: 0.9, 1: 0.09999999999999998}
Test accuracy : 0.8571428571428571


Predict,0,1
Actual,,
0,371,9
1,54,7


{0: 0.99, 1: 0.010000000000000009}
Test accuracy : 0.8594104308390023


Predict,0,1
Actual,,
0,377,3
1,59,2


In [177]:
tuning_results

,class_0_weight,class_1_weight,train_accuracy,tesst_accuracy,precision_class_0,precision_class_1,precision_overall,recall_calss_0,recall_class_1,recall_overall,test_accuracy
0,0.01,0.99,0.3421,0.2721,0.92,0.15,0.81,0.17,0.90,0.27,0.2721
1,0.10,0.90,0.8056,0.7324,0.93,0.29,0.85,0.74,0.67,0.73,0.7324
2,0.20,0.80,0.8707,0.8299,0.90,0.39,0.83,0.90,0.41,0.83,0.8299
3,0.30,0.70,0.8814,0.8390,0.91,0.42,0.84,0.91,0.41,0.84,0.8390
4,0.40,0.60,0.8941,0.8322,0.90,0.38,0.83,0.91,0.34,0.83,0.8322
5,0.50,0.50,0.8960,0.8458,0.88,0.39,0.81,0.95,0.20,0.85,0.8458
6,0.60,0.40,0.8941,0.8617,0.88,0.50,0.82,0.98,0.13,0.86,0.8617
7,0.70,0.30,0.8902,0.8639,0.88,0.53,0.83,0.98,0.16,0.86,0.8639
8,0.80,0.20,0.8824,0.8526,0.87,0.38,0.80,0.97,0.10,0.85,0.8526
9,0.90,0.10,0.8698,0.8571,0.87,0.44,0.81,0.98,0.11,0.86,0.8571


Class 0의 가중치가 커질수록, class 0으로 더 많이 예측한다. 우선 예측량이 많아지기 때문에 리콜은 상대적으로 높다. 정확도는 떨어질지라도 class 0으로 예측하는 양이 많아지기 때문에 recall이 상대적으로 높아지기 때문이다. 하지만 예측하는 양이 상대적으로 많이지기 때문에 precision은 떨어지게 된다.

결과를 살펴보면, class 0의 가중치가 0.3일 때, accuracy 및 precision, recall 이 괜찮은 결과를 보이고 있다.